In [ ]:
# Import python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ML Libraries
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import time
from snowflake.ml.experiment import ExperimentTracking

# Feature store
from snowflake.ml.feature_store import FeatureStore
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Model registry
from snowflake.ml.registry import Registry
registry = Registry(session=session, database_name="dev", schema_name="data_science")

# Initialize Experiment Tracking
exp = ExperimentTracking(session=session)
exp.set_experiment("User_activity_forecasting_models")

#### Get training dataset from FS

In [ ]:
# Connect to existing Feature Store
fs = FeatureStore(
    session=session,
    database="dev",
    name="user_activity_feature_store",
    default_warehouse="analytics_wh_flex"
)

In [ ]:
# Retrieve required feature views
fv_ads = fs.get_feature_view(name="user_features_ads", version="1.2")
fv_call_quality = fs.get_feature_view(name="user_features_call_quality", version="1.2")
fv_call_rating = fs.get_feature_view(name="user_features_call_rating", version="1.2")
fv_data_usage = fs.get_feature_view(name="user_features_data_usage", version="1.2")
fv_drawer_event = fs.get_feature_view(name="user_features_drawer_event", version="1.2")
fv_inbound_calls = fs.get_feature_view(name="user_features_inbound_calls", version="1.2")
fv_messages = fs.get_feature_view(name="user_features_messages", version="1.2")
fv_nps_rating = fs.get_feature_view(name="user_features_nps_rating", version="1.2")
fv_outbound_calls = fs.get_feature_view(name="user_features_outbound_calls", version="1.2")
fv_redial = fs.get_feature_view(name="user_features_redial", version="1.2")
fv_sessions = fs.get_feature_view(name="user_features_sessions", version="1.2")

all_features = [
    fv_ads,
    fv_call_quality, 
    fv_call_rating,
    fv_data_usage,
    fv_drawer_event,
    fv_inbound_calls,
    fv_messages,
    fv_nps_rating,
    fv_outbound_calls,
    fv_redial,
    fv_sessions  
]

In [ ]:
# spine df
spine_df = session.sql("""
WITH daily AS (
    SELECT username, date_utc, SUM(time_in_app_mins_per_day) AS mins
    FROM metrics_daily_userlevel_app_time_sessions
    WHERE date_utc >= DATEADD('day', -37, '2025-11-19')
    GROUP BY username, date_utc
),
labeled AS (
    SELECT up.user_id_hex, d.date_utc,
        COALESCE(SUM(IFF(d.mins > 1, 1, 0)) OVER (
            PARTITION BY d.username ORDER BY d.date_utc
            RANGE BETWEEN INTERVAL '1 DAY' FOLLOWING AND INTERVAL '7 DAYS' FOLLOWING
        ), 0) AS active_days_in_week
    FROM daily d
    JOIN user_profiles up ON d.username = up.latest_username
),
filtered AS (
    SELECT * FROM labeled
    WHERE date_utc <= DATEADD('day', -7, '2025-11-19')
),
sampled AS (
    SELECT *, ROW_NUMBER() OVER (
        PARTITION BY date_utc 
        ORDER BY HASH(user_id_hex || date_utc)
    ) AS rn
    FROM filtered
)
SELECT user_id_hex, date_utc, active_days_in_week 
FROM sampled 
WHERE rn <= 70000
""")

In [ ]:
#Generate training dataset
df = fs.generate_training_set(
    spine_df=spine_df,
    features=all_features,
    spine_label_cols="active_days_in_week",
)

In [ ]:
# Optional: Save df as a table to avoid rerun cost
# df.write.mode("overwrite").save_as_table("dev.data_science.training_data")

#### Training

In [ ]:
# to_pandas
df = df.to_pandas().fillna(0)
print(f"Dataset shape: {df.shape}")
df.head()

## Data Exploration

In [ ]:
# Basic data exploration
pd.set_option('display.max_columns', None)

print("Data Types:")
print(df.dtypes)

print("\nStatistics of Numerical Columns:")
print(df.describe())

print("\nMissing Values:")
print(df.isnull().sum().sum())

In [ ]:
# Target distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(df['ACTIVE_DAYS_IN_WEEK'], bins=8, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Active Days in Next 7 Days')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Target Distribution')

target_counts = df['ACTIVE_DAYS_IN_WEEK'].value_counts().sort_index()
axes[1].bar(target_counts.index, target_counts.values / len(df) * 100, color='coral', edgecolor='black')
axes[1].set_xlabel('Active Days')
axes[1].set_ylabel('Percentage (%)')
axes[1].set_title('Target Distribution (Percentage)')

plt.tight_layout()
plt.show()

print(f"Target Mean: {df['ACTIVE_DAYS_IN_WEEK'].mean():.2f}")
print(f"Target Std: {df['ACTIVE_DAYS_IN_WEEK'].std():.2f}")

In [ ]:
# Correlation with target
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_with_target = df[numeric_cols].corr()['ACTIVE_DAYS_IN_WEEK'].abs().sort_values(ascending=False)

print("Top 15 Features Correlated with Target:")
print("=" * 50)
for feat, corr in corr_with_target.head(16).items():
    if feat != 'ACTIVE_DAYS_IN_WEEK':
        print(f"{feat}: {corr:.4f}")

# Correlation heatmap for top features
top_features = corr_with_target.head(12).index.tolist()
top_corr = df[top_features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(top_corr, annot=True, fmt='.2f', cmap='coolwarm', center=0, square=True)
plt.title('Top Features - Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Split training and testing dataset using temporal split (by date)
X = df.drop(['USER_ID_HEX', 'DATE_UTC', 'ACTIVE_DAYS_IN_WEEK'], axis=1)
y = df['ACTIVE_DAYS_IN_WEEK']

# Temporal split: 80% of days for training, 20% for testing
dates = df['DATE_UTC']
unique_dates = sorted(dates.unique())
n_train_days = int(len(unique_dates) * 0.8)
cutoff_date = unique_dates[n_train_days - 1]

train_mask = dates <= cutoff_date
test_mask = dates > cutoff_date

X_train = X[train_mask].reset_index(drop=True)
X_test = X[test_mask].reset_index(drop=True)
y_train = y[train_mask].reset_index(drop=True)
y_test = y[test_mask].reset_index(drop=True)

print(f"Train: {len(X_train)} rows ({n_train_days} days, <= {cutoff_date})")
print(f"Test:  {len(X_test)} rows ({len(unique_dates) - n_train_days} days, > {cutoff_date})")

In [ ]:
# Preprocessing
numerical_cols = X.select_dtypes(include=['number']).columns
preprocess=ColumnTransformer([
    ("num", StandardScaler(), numerical_cols)
])

In [ ]:
# Baseline models
baseline_models = [
    # Linear models
    ("LinearRegression_BaseLine", LinearRegression()),
    ("Ridge_BaseLine", Ridge()),
    ("Lasso_BaseLine", Lasso()),
    
    # Random Forest variations
    ("RandomForest_BaseLine", RandomForestRegressor()),
    
    # Gradient Boosting
    ("GradientBoosting_BaseLine", GradientBoostingRegressor()),
    
    # XGBoost
    ("XGBoost_BaseLine", XGBRegressor(verbosity=0)),
    
    # LightGBM
    ("LightGBM_BaseLine", LGBMRegressor())
]

In [ ]:
# Training baseline models
for name, model in baseline_models:
    print(f"Training baseline: {name}")

    pipeline = Pipeline(steps=[
        ("preprocess", preprocess),
        ("model", model)
    ])

    # Start timer
    start_time= time.time()

    # Fit on training data
    pipeline.fit(X_train, y_train)

    # Calculate training time
    training_time = time.time() - start_time

    #Predict on test data
    y_pred =  pipeline.predict(X_test)

    #Evaluation RMSE, MAE, R2
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)

    # Get sizes
    n_train = len(X_train)
    n_test= len(X_test)
    n_features = X_train.shape[1]

    print(f"{name} -> Test RMSE: {rmse:.4f}")

    #Log experiment
    with exp.start_run():
        exp.log_metric("rmse", rmse)
        exp.log_metric("mae", mae)
        exp.log_metric("training_time_seconds", training_time)
        exp.log_param("model_type", name)
        exp.log_param("n_train_samples", n_train)
        exp.log_param("n_test_samples", n_test)
        exp.log_param("n_features", n_features)
        exp.log_model(model_name=f"{name}_model", model=pipeline, sample_input_data= X_train.head())
        
    pipeline = None